<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/Dengue_Supervised_Project/3_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Optimization

To have better results with the models, next thing is to optimize in order to improve the performance.


## 3.1  Initialization

First of all, it is going to be imported all the Python libraries normally used in data science. Also, it will be imported the datasets.

In [1]:
# data load and manipulation
from google.colab import files
from google.colab import drive
import io
# dataframes library
import pandas as pd
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
# basic operations
import numpy as np
# cross validation
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
# feature selection
from sklearn.feature_selection import SelectKBest, chi2, f_regression
# regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
# metrics 
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr
# normalization
from sklearn.preprocessing import MinMaxScaler
# outliers
from scipy import stats

This function is used por obtain a dataframe from a csv file, in which we can determine the columns we want.

In [2]:
def upload_files (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

Importing the preprocessed train and test dataset.

In [3]:
train = upload_files([])
train.head()

Saving processed_train.csv to processed_train.csv
User uploaded file "processed_train.csv" with length 291364 bytes


,city,year,weekofyear,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month
0,0,1990,18,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,4
1,0,1990,19,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,5
2,0,1990,20,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,5
3,0,1990,21,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,5
4,0,1990,22,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,5


In [4]:
test = upload_files([])
test.head()

Saving processed_test.csv to processed_test.csv
User uploaded file "processed_test.csv" with length 82243 bytes


,city,year,weekofyear,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,month
0,0,2008,18,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2,4
1,0,2008,19,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3,5
2,0,2008,20,-0.016133,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0,5
3,0,2008,21,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3,5
4,0,2008,22,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1,5


In [5]:
original_test = upload_files([])
original_test.head()

Saving dengue_features_test.csv to dengue_features_test.csv
User uploaded file "dengue_features_test.csv" with length 82465 bytes


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


## 3.2 Optimizing data

Just like with the baseline, outliers are going to be removed from the data to avoid noise.

In [6]:
train.shape

(1454, 24)

In [7]:
# train = train[(np.abs(stats.zscore(train.drop(['year','weekofyear','total_cases'],axis=1))) < 5).all(axis=1)]

In [8]:
train.shape

(1454, 24)

The same with the binarization of **month** feature, that it is a line of improvemnt that it was introduce in the baseline.

In [9]:
train = pd.get_dummies(train, columns=['month'])
test = pd.get_dummies(test, columns=['month'])

Other line of improvement will be having two models for each city, and see if the result of the predicted data is more accurate for each of the cities. 

In [10]:
train_sj = train[train['city']==0]
train_iq = train[train['city']==1]

In [11]:
test_sj = test[test['city']==0]
test_iq = test[test['city']==1]

With this configuration, **city** column is not necessary for the train and test data of San Juan and Iquitos.

In [12]:
train_sj = train_sj.drop(['city'], axis=1)
train_iq = train_iq.drop(['city'], axis=1)

In [13]:
test_sj = test_sj.drop(['city'], axis=1)
test_iq = test_iq.drop(['city'], axis=1)

## 3.2 Gradient Boosting Optimization

First, it is going to be defined a function that it will return the result of a Randomized Search of Gradient Boosting.

In [14]:
def get_best_boosting(X_train, y_train):
  '''param_dist = {'n_estimators': [4, 8, 16, 32, 64, 128, 150], # Number of trees in random forest
                'max_features': ['auto', 'sqrt'], # Number of features to consider at every split
                'max_depth': [8, 4, 2, None], # Maximum number of levels in tree
                'min_samples_split': [2, 4, 8, 16, 32, 64, 128, 256], #  Minimum number of samples required to split a node
                'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64, 128, 256], # Minimum number of samples required at each leaf node
                'bootstrap': [True, False], # Method of selecting samples for training each tree
                "criterion": ["mse", "mae"]
  }'''
  param_dist = {'n_estimators': np.arange(100, 250, 10), # Number of trees in random forest
                'max_features': ['auto', 'sqrt'], # Number of features to consider at every split
                'max_depth': [16, 8, 4, 2, None], # Maximum number of levels in tree
                'min_samples_split': [2, 4, 8, 16, 32], #  Minimum number of samples required to split a node
                'min_samples_leaf': [1, 2, 4, 8, 16], # Minimum number of samples required at each leaf node
                'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3], # Method of selecting samples for training each tree
                'loss': ['ls', 'lad', 'huber', 'quantile'],
                'criterion': ['mse', 'friedman_mse', 'mae']
  }
  boost = GradientBoostingRegressor()
  random_regres = RandomizedSearchCV(estimator = boost, param_distributions = param_dist, 
                               n_iter = 100, cv = 5, random_state=0, n_jobs = -1)
  
  # Fit the random search model
  random_regres.fit(X = X_train, y = y_train)
  
  return random_regres

In [15]:
def report(results, n_top=3): 
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

### 3.2.1 San Juan model

The optimization for finding the best **San Juan** models is going to be executed. First, it is going to be choose the best features. The extraction of the features is done by the execution of Gradient Boosting of the data of this city.

In [18]:
selected_feat_boost = ['reanalysis_air_temp_k', 'ndvi_sw', 
                       'reanalysis_dew_point_temp_k', 'reanalysis_min_air_temp_k', 
                       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_precip_amt_kg_per_m2', 'ndvi_nw',
                       'station_precip_mm', 'reanalysis_max_air_temp_k', 'station_avg_temp_c']

In [19]:
X_train_sj = train_sj.drop(['year', 'total_cases'], axis=1)[selected_feat_boost]
X_test_sj = test_sj.drop(['year'], axis=1)[selected_feat_boost]
y_train_sj = train_sj['total_cases']

After that, the models with the parameters that obtain the best scores for **San Juan** city.

In [ ]:
best_boosts_sj = get_best_boosting(X_train_sj, y_train_sj)

In [ ]:
report(best_boosts_sj.cv_results_, n_top = 5)

The model with the best score is clearly better than the others, so it will be selected the model in the rank 1.

In [ ]:
boost_sj = best_boosts_sj.best_estimator_

In [ ]:
boost_sj.fit(X = X_train_sj, y = y_train_sj)
y_pred_sj = boost_sj.predict(X_test_sj)

In [ ]:
y_pred_sj = np.rint(y_pred_sj)
y_pred_sj = y_pred_sj.astype(int)
results_sj = np.hstack(y_pred_sj)

### 3.2.2 Iquitos model

The optimization for finding the best **Iquitos** models is going to be executed. First, it is going to be choose the best features. Like the other city, the features were extracted from the execution of the data of the proper city.

In [ ]:
selected_feat_boost = ['reanalysis_air_temp_k', 'ndvi_sw', 
                       'reanalysis_dew_point_temp_k', 'reanalysis_min_air_temp_k', 
                       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_precip_amt_kg_per_m2', 'ndvi_nw',
                       'station_precip_mm', 'reanalysis_max_air_temp_k', 'station_avg_temp_c']

In [ ]:
X_train = train_iq.drop(['year', 'total_cases'], axis=1)[selected_feat_boost]
X_test = test_iq.drop(['year'], axis=1)[selected_feat_boost]
y_train = train_iq['total_cases']

After that, the models with the parameters that obtain the best scores for **San Juan** city.

In [ ]:
best_boosts = get_best_boosting(X_train, y_train)

In [ ]:
report(best_boosts.cv_results_, n_top = 5)

The model selected will be the best one.

In [ ]:
boost = best_boosts.best_estimator_

In [ ]:
boost.fit(X = X_train, y = y_train)
y_pred = boost.predict(X_test)

In [ ]:
y_pred = np.abs(np.rint(y_pred))
y_pred = y_pred.astype(int)
results = np.hstack(y_pred)

In [ ]:
results_iq = results[train_iq.shape[0]:]

### 3.2.3 Submission of the two models

In [ ]:
results = np.append(results_sj, results_iq)

In [ ]:
results

In [ ]:
output = pd.DataFrame({ 'city': original_test['city'], 'year': original_test['year'], 
                       'weekofyear': original_test['weekofyear'], 'total_cases': results})

with open('submission_optimization_10.csv', 'w') as f:
  output.to_csv(f,  index = False)

files.download('submission_optimization_10.csv')

output.head()

In [ ]:
output[output['city'] == 'iq']